In [1]:
import pandas as pd
from sklearn.ensemble.partial_dependence import partial_dependence, plot_partial_dependence

In [50]:
# Loading the training data
train_data = pd.read_csv('train.csv')

#Loading the test data
test_data = pd.read_csv('test.csv')
test_data=test_data.rename(columns={'actual _credit_score':'actual_credit_score'})

y=train_data.total_sales

# Dropping the 'Criminal' columm from training data
X_train = train_data.drop('total_sales',axis=1)

# Columns list in training data
columns_list = X_train.columns

# Check if columns had any missing values
columns_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]

# Columns does not have any missing values so we don't use imputer
print(columns_with_missing)

# Check if training file has any categorical data
non_categorical_predictors = X_train.select_dtypes(include=['object'])
#print(non_categorical_predictors)

[]


In [56]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor

one_hot_encoded_training_predictors = pd.get_dummies(training_predictors)

def get_mae(X, y):
    # multiple by -1 to make positive MAE score instead of neg value returned as sklearn convention
    return -1 * cross_val_score(RandomForestRegressor(50), 
                                X, y, 
                                scoring = 'neg_mean_absolute_error').mean()

predictors_without_categoricals = training_predictors.select_dtypes(exclude=['object'])
mae_one_hot_encoded = get_mae(one_hot_encoded_training_predictors, y)

print('Mean Abslute Error with One-Hot Encoding: ' + str(int(mae_one_hot_encoded)))

ValueError: Found input variables with inconsistent numbers of samples: [344, 44]

In [59]:
cardinality_cols = [cname for cname in X_train.columns 
                    if X_train[cname].nunique()<10 and X_train[cname].dtype == "object"]
numerical_cols = [cname for cname in X_train.columns 
                    if X_train[cname].dtype in ['int64', 'float64']]

my_columns =cardinality_cols + numerical_cols
drop_column=['outlet_no','zip','avg_age','blue_collar','female','male','latitude','longitude','employee_size',
             'actual_credt_score']
print(my_columns)
training_predictors = X_train[my_columns]
testing_predictors = test_data[my_columns]

one_hot_encoded_training_predictors = pd.get_dummies(training_predictors)
one_hot_encoded_test_predictors = pd.get_dummies(testing_predictors)

final_train, final_test = one_hot_encoded_training_predictors.align(one_hot_encoded_test_predictors,
                                                                    join='left', 
                                                                    axis=1)

['store_location', 'time_zone', 'credit_score', 'credit_score_range', 'outlet_no', 'business_type', 'zip', 'avg_age', 'blue_collar', 'white_collar', 'female', 'male', 'total_household_size ', 'total_household_income', 'latitude', 'longitude', 'employee_size', 'actual_credit_score']


In [41]:
from sklearn.ensemble import RandomForestRegressor
my_model = RandomForestRegressor()
my_model.fit(final_train, y)

predicted_prices = my_model.predict(final_test)
# We will look at the predicted prices to ensure we have something sensible.
value=predicted_prices.astype(int)
print(value)


[  4459    476    501    705    774   1367   1610    524    512   1436
    770   4242   1438   1821    507    508   1499   1286   1826   4816
   1612    542   2311    516   1659   2199    811   6650   1639    693
    491    916 103479   1514   3435   3933   1382   1846   4013   9472
    511  14130   1720   1691   3303   4752   1933   1681   5945   1428
 108253  11212    765  16493   1420   1470   1805   3769    543    943
    727  16983  20109   5362   1982  12631  21034   1392   7817   7312
  16529   8146   5730    850  11060   3533   7442  11460   3601  10499
   1640   3134   9926    579   3885  17877  37997    525   6379  14530
  15854    731    534    631    554    656    796    725   1035   1600
    512    531  10648   1477   1655    692   1233  76131  12173 314762
   9004   1602   1914   7579   9025   7787    825    555 755591   1243
    533  11302   1393  10334   4760    462   1422    522    535   3126
  21750   2204    512    523   1595   1633   1831   1810   1704   1724
   104

In [42]:
my_submission = pd.DataFrame({'outlet_no': test_data.outlet_no, 'total_sales_actual': value})
# you could use any filename. We choose submission here
my_submission.to_csv('submission.csv', index=False)

In [51]:
from sklearn.svm import LinearSVC
from sklearn.feature_selection import RFE
from sklearn import datasets
dataset = datasets.load_iris()
svm = RandomForestRegressor()# create the RFE model for the svm classifier 
# and select attributes    
rfe = RFE(svm, 10) #71
rfe = rfe.fit(final_train, y)
# print summaries for the selection of attributes
print(rfe.support_)
print(rfe.ranking_)

[ True False  True  True  True False  True  True False False  True  True
  True  True False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False]
[ 1 25  1  1  1  2  1  1  4  3  1  1  1  1  5 21 26 32 28 19  9 18 15  6
 24 16 12 14 17  8 27 13 29 33 34 30 22 20  7 11 10 23 35 31]


In [52]:
y=final_train.columns
for i in range(0,len(y)):
    print(rfe.support_[i],rfe.ranking_[i],y[i])

drop_column=['outlet_no','zip','avg_age','blue_collar','female','male','latitude','longitude','employee_size',
             'actual_credt_score']

True 1 outlet_no
False 25 business_type
True 1 zip
True 1 avg_age
True 1 blue_collar
False 2 white_collar
True 1 female
True 1 male
False 4 total_household_size 
False 3 total_household_income
True 1 latitude
True 1 longitude
True 1 employee_size
True 1 actual_credit_score
False 5 store_location_AT-WORK
False 21 store_location_EATING & DRINKING
False 26 store_location_EDUCATIONAL
False 32 store_location_ENTERTAINMENT/RECREATION/LEISURE
False 28 store_location_GROCERY SHOPPING
False 19 store_location_OTHER SHOPPING & SERVICES
False 9 store_location_THIRD PARTY (NON-CONSUMER)
False 18 store_location_TRAVEL/TRANSPORTATION/HOSPITALITY
False 15 time_zone_CST
False 6 time_zone_EST
False 24 time_zone_MST
False 16 time_zone_PST
False 12 credit_score_A
False 14 credit_score_A+
False 17 credit_score_B
False 8 credit_score_B+
False 27 credit_score_C
False 13 credit_score_C+
False 29 credit_score_I
False 33 credit_score_P
False 34 credit_score_U
False 30 credit_score_range_70 TO 74
False 22 credit